# Simulating Upcoming Season

In this notebook I will get odds for the upcoming season and simulate the upcoming season many times based on the odds.

In [1]:
import numpy as np
import random

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("MY_API_KEY")
sport = 'americanfootball_nfl'
regions = 'us'
markets = 'h2h'
oddsFormat = 'decimal'

In [49]:
import requests

base_url = f'https://api.the-odds-api.com/v4/sports/{sport}/odds/'

# Query parameters as a dictionary
params = {
    'apiKey': api_key,
    'regions': regions,
    'markets': markets,
    'oddsFormat': oddsFormat
}

response = requests.get(base_url, params=params)


In [50]:
if response.status_code == 200:
    data = response.json()  # parse JSON response
    print('Success:', data[:1])  # print first item as preview
else:
    print('Error:', response.status_code, response.text)

Success: [{'id': 'f1bc532dff946d15cb85654b5c4b246e', 'sport_key': 'americanfootball_nfl', 'sport_title': 'NFL', 'commence_time': '2025-09-05T00:21:00Z', 'home_team': 'Philadelphia Eagles', 'away_team': 'Dallas Cowboys', 'bookmakers': [{'key': 'draftkings', 'title': 'DraftKings', 'last_update': '2025-07-27T19:24:09Z', 'markets': [{'key': 'h2h', 'last_update': '2025-07-27T19:24:08Z', 'outcomes': [{'name': 'Dallas Cowboys', 'price': 3.7}, {'name': 'Philadelphia Eagles', 'price': 1.29}]}]}, {'key': 'lowvig', 'title': 'LowVig.ag', 'last_update': '2025-07-27T19:24:08Z', 'markets': [{'key': 'h2h', 'last_update': '2025-07-27T19:24:08Z', 'outcomes': [{'name': 'Dallas Cowboys', 'price': 3.3}, {'name': 'Philadelphia Eagles', 'price': 1.36}]}]}, {'key': 'betonlineag', 'title': 'BetOnline.ag', 'last_update': '2025-07-27T19:24:42Z', 'markets': [{'key': 'h2h', 'last_update': '2025-07-27T19:24:42Z', 'outcomes': [{'name': 'Dallas Cowboys', 'price': 3.3}, {'name': 'Philadelphia Eagles', 'price': 1.36}]}

In [51]:
import pandas as pd

games = data

# Flatten
rows = []
for game in games:
    game_time = game['commence_time']
    home_team = game['home_team']
    away_team = game['away_team']

    for bookmaker in game['bookmakers']:
        bookmaker_name = bookmaker['title']
        for market in bookmaker['markets']:
            if market['key'] == 'h2h':  # focus on moneyline
                for outcome in market['outcomes']:
                    team = outcome['name']
                    odds = outcome['price']
                    rows.append({
                        'Game Start': game_time,
                        'Home Team': home_team,
                        'Away Team': away_team,
                        'Bookmaker': bookmaker_name,
                        'Team': team,
                        'Odds': odds
                    })
            

odds_df = pd.DataFrame(rows)
odds_df.head()



,Game Start,Home Team,Away Team,Bookmaker,Team,Odds
0,2025-09-05T00:21:00Z,Philadelphia Eagles,Dallas Cowboys,DraftKings,Dallas Cowboys,3.70
1,2025-09-05T00:21:00Z,Philadelphia Eagles,Dallas Cowboys,DraftKings,Philadelphia Eagles,1.29
2,2025-09-05T00:21:00Z,Philadelphia Eagles,Dallas Cowboys,LowVig.ag,Dallas Cowboys,3.30
3,2025-09-05T00:21:00Z,Philadelphia Eagles,Dallas Cowboys,LowVig.ag,Philadelphia Eagles,1.36
4,2025-09-05T00:21:00Z,Philadelphia Eagles,Dallas Cowboys,BetOnline.ag,Dallas Cowboys,3.30


In [33]:
odds_df['Avg Odds'] = odds_df.groupby(
    ['Game Start', 'Home Team', 'Away Team', 'Team']
    )['Odds'].transform('mean')
odds_df['Game Key'] = odds_df['Game Start'] + '|' + odds_df['Home Team'] + '|' + odds_df['Away Team']
odds_df['Join Key'] = odds_df['Home Team'] + '|' + odds_df['Away Team']
avg_odds_df = odds_df.groupby(
    ['Game Start', 'Home Team', 'Away Team', 'Team', 'Game Key', 'Join Key']
    )['Odds'].mean().reset_index(name='Avg Odds')

In [34]:
avg_odds_df = avg_odds_df[['Game Start', 'Team', 'Avg Odds', 'Game Key', 'Join Key']].rename(columns={'Avg Odds': 'Odds'})

In [35]:
avg_odds_df['Implied Prob'] = 1 / avg_odds_df['Odds']
probs = []
for gk in np.unique(avg_odds_df['Game Key']):
    this_game = avg_odds_df[avg_odds_df['Game Key'] == gk] 
    p = list(this_game['Implied Prob'] / np.sum(this_game['Implied Prob']))
    probs.extend(p)
avg_odds_df['Win Prob'] = probs


In [36]:
avg_odds_df.head()

,Game Start,Team,Odds,Game Key,Join Key,Implied Prob,Win Prob
0,2025-09-05T00:21:00Z,Dallas Cowboys,3.47375,2025-09-05T00:21:00Z|Philadelphia Eagles|Dalla...,Philadelphia Eagles|Dallas Cowboys,0.287873,0.275736
1,2025-09-05T00:21:00Z,Philadelphia Eagles,1.32250,2025-09-05T00:21:00Z|Philadelphia Eagles|Dalla...,Philadelphia Eagles|Dallas Cowboys,0.756144,0.724264
2,2025-09-06T00:01:00Z,Kansas City Chiefs,1.70875,2025-09-06T00:01:00Z|Los Angeles Chargers|Kans...,Los Angeles Chargers|Kansas City Chiefs,0.585223,0.559742
3,2025-09-06T00:01:00Z,Los Angeles Chargers,2.17250,2025-09-06T00:01:00Z|Los Angeles Chargers|Kans...,Los Angeles Chargers|Kansas City Chiefs,0.460299,0.440258
4,2025-09-07T17:01:00Z,Atlanta Falcons,2.04250,2025-09-07T17:01:00Z|Atlanta Falcons|Tampa Bay...,Atlanta Falcons|Tampa Bay Buccaneers,0.489596,0.468445


In [37]:
avg_odds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Game Start    514 non-null    object 
 1   Team          514 non-null    object 
 2   Odds          514 non-null    float64
 3   Game Key      514 non-null    object 
 4   Join Key      514 non-null    object 
 5   Implied Prob  514 non-null    float64
 6   Win Prob      514 non-null    float64
dtypes: float64(3), object(4)
memory usage: 28.2+ KB


In [38]:
# there are 272 regular season games in 2025, 544 is double this with 1 row for each team involved.

In [39]:
def simulate_game(win_probs):
    rand = np.random.rand()
    winner_idx = 0 if rand < win_probs.iloc[0] else 1
    return [True if i == winner_idx else False for i in range(len(win_probs))]

In [40]:
num_sims = 10000
sim_dfs = []
for i in range(num_sims):
    new_df = avg_odds_df.copy()
    new_df['Won?'] = new_df.groupby('Game Key')['Win Prob'].transform(simulate_game)
    new_df['Season'] = [i + 1 for j in range(len(new_df))]
    sim_dfs.append(new_df)

simmed_seasons = pd.concat(sim_dfs)[['Season', 'Game Start','Team', 'Odds', 'Game Key','Join Key', 'Implied Prob', 'Won?']]

In [41]:
simmed_seasons.head()

,Season,Game Start,Team,Odds,Game Key,Join Key,Implied Prob,Won?
0,1,2025-09-05T00:21:00Z,Dallas Cowboys,3.47375,2025-09-05T00:21:00Z|Philadelphia Eagles|Dalla...,Philadelphia Eagles|Dallas Cowboys,0.287873,False
1,1,2025-09-05T00:21:00Z,Philadelphia Eagles,1.32250,2025-09-05T00:21:00Z|Philadelphia Eagles|Dalla...,Philadelphia Eagles|Dallas Cowboys,0.756144,True
2,1,2025-09-06T00:01:00Z,Kansas City Chiefs,1.70875,2025-09-06T00:01:00Z|Los Angeles Chargers|Kans...,Los Angeles Chargers|Kansas City Chiefs,0.585223,False
3,1,2025-09-06T00:01:00Z,Los Angeles Chargers,2.17250,2025-09-06T00:01:00Z|Los Angeles Chargers|Kans...,Los Angeles Chargers|Kansas City Chiefs,0.460299,True
4,1,2025-09-07T17:01:00Z,Atlanta Falcons,2.04250,2025-09-07T17:01:00Z|Atlanta Falcons|Tampa Bay...,Atlanta Falcons|Tampa Bay Buccaneers,0.489596,True


In [42]:
simmed_seasons.tail()

,Season,Game Start,Team,Odds,Game Key,Join Key,Implied Prob,Won?
509,10000,2026-01-04T18:00:00Z,Seattle Seahawks,2.70,2026-01-04T18:00:00Z|San Francisco 49ers|Seatt...,San Francisco 49ers|Seattle Seahawks,0.370370,False
510,10000,2026-01-04T18:00:00Z,Carolina Panthers,3.50,2026-01-04T18:00:00Z|Tampa Bay Buccaneers|Caro...,Tampa Bay Buccaneers|Carolina Panthers,0.285714,False
511,10000,2026-01-04T18:00:00Z,Tampa Bay Buccaneers,1.32,2026-01-04T18:00:00Z|Tampa Bay Buccaneers|Caro...,Tampa Bay Buccaneers|Carolina Panthers,0.757576,True
512,10000,2026-01-04T21:05:00Z,Denver Broncos,1.74,2026-01-04T21:05:00Z|Denver Broncos|Los Angele...,Denver Broncos|Los Angeles Chargers,0.574713,False
513,10000,2026-01-04T21:05:00Z,Los Angeles Chargers,2.14,2026-01-04T21:05:00Z|Denver Broncos|Los Angele...,Denver Broncos|Los Angeles Chargers,0.467290,True


In [43]:
schedule_2025 = pd.read_csv("../data/schedule_2025.csv")
schedule_2025 = schedule_2025[schedule_2025['Week'].isin([str(week) for week in np.arange(1, 19)])]
schedule_2025.head()

,Unnamed: 0,Week,Day,Unnamed: 2,VisTm,Pts,Unnamed: 5,HomeTm,Pts.1,Time
52,52,1,Thu,September 4,Dallas Cowboys,NaN,@,Philadelphia Eagles,NaN,8:20 PM
53,53,1,Fri,September 5,Kansas City Chiefs,NaN,@,Los Angeles Chargers,NaN,8:00 PM
54,54,1,Sun,September 7,Tampa Bay Buccaneers,NaN,@,Atlanta Falcons,NaN,1:00 PM
55,55,1,Sun,September 7,Cincinnati Bengals,NaN,@,Cleveland Browns,NaN,1:00 PM
56,56,1,Sun,September 7,Miami Dolphins,NaN,@,Indianapolis Colts,NaN,1:00 PM


In [44]:
schedule_2025['Join Key'] = schedule_2025['HomeTm'] + '|' + schedule_2025['VisTm']

In [45]:
merged = pd.merge(simmed_seasons, schedule_2025, on='Join Key')

In [46]:
merged[['Season', 'Week', 'Game Start','Team', 'Odds', 'Implied Prob', 'Won?']].to_csv('../data/simulated_upcoming_season')